In [2]:
import tensorflow as tf
import numpy as np

import os

# Text Preprocessing
import re
import string
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [3]:
# create training and validation data
batch_size = 64
main_path = 'D:/keras_datasets/aclImdb'
train_path = os.path.join(main_path, 'train')
test_path = os.path.join(main_path, 'test')

raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    train_path,
    batch_size = batch_size,
    validation_split = .2,
    subset = 'training',
    seed = 1337   # always set a seed for reproducibility
)

raw_val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    train_path,
    batch_size = batch_size,
    validation_split = 0.2,
    subset = 'validation',
    seed = 1337 # always set a seed for reproducibility
)

raw_test_ds = tf.keras.preprocessing.text_dataset_from_directory(
    test_path, batch_size = batch_size
)

Found 75000 files belonging to 3 classes.
Using 60000 files for training.
Found 75000 files belonging to 3 classes.
Using 15000 files for validation.
Found 25000 files belonging to 2 classes.


### Preprocessing the data

In [4]:
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data) # built-in tf function to operate on tesors
    stripped_html = tf.strings.regex_replace(lowercase, "<br />", " ")
    return tf.strings.regex_replace(stripped_html, "[%s]" % re.escape(string.punctuation), "")

max_features = 20000
embedding_dim = 128
sequence_length = 500

vectorization = TextVectorization(
    standardize = custom_standardization,
    max_tokens = max_features,
    output_mode = 'int',
    output_sequence_length = sequence_length
)

# to use the vectorization layer, the function adapt can be called
# for example : vectorization.adapt(text_input)

In [5]:
text_ds = raw_train_ds.map(lambda x, y : x)
vectorization.adapt(text_ds)

KeyboardInterrupt: 